# Import required libraries

In [1]:
# import basic libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# import libraries for machine learning models
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier

# import libraries to solve LP
from pulp import *

# Output $p_i$ and $q_i$ for each candidate i $\in$ [n]

### Train the predict algorithm

In [154]:
# read data
df = pd.read_csv('clean_law_school.csv', index_col = 0)

# split data into training and testing part
target = ['admit', 'enroll']
y = df[target]
X = df.drop(target, axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, shuffle = True, random_state = 1)

# implement the machine learning model to predict p_i and q_i
lg_clf = ClassifierChain(RandomForestClassifier())
lg_clf.fit(X_train, y_train)

ClassifierChain(classifier=RandomForestClassifier(), require_dense=[True, True])

### Output $p_i$ and $q_i$ for each i in the test data

In [155]:
y_pred = lg_clf.predict_proba(X_test)
X_test = pd.merge(X_test, y_test, left_index = True, right_index = True)
X_test[['p_i', 'q_i']] = np.round(y_pred.toarray(), 3)
X_test

,lsat,gpa,resident,gender,black,hispanic,asian,white,other_race,c_admit_rate,c_enroll_rate,es,admit,enroll,p_i,q_i
54066,150.0,2.77,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.649,0.381,0.323,0.0,0.0,0.070,0.000
59441,155.0,2.81,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.000,0.285,0.151,0.0,0.0,0.010,0.000
11354,161.0,3.78,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.219,0.180,0.108,1.0,0.0,0.489,0.000
68562,164.0,3.44,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.361,0.149,0.387,1.0,0.0,0.730,0.063
97741,149.0,3.79,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.642,0.405,0.280,0.0,0.0,0.090,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51882,153.0,3.05,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.649,0.381,0.323,0.0,0.0,0.070,0.000
62536,162.0,3.57,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000,0.285,0.151,1.0,1.0,0.478,0.000
98995,156.0,3.30,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.638,0.740,0.500,1.0,1.0,0.972,0.948
6304,152.0,3.22,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.693,1.000,0.301,0.0,0.0,0.023,0.000


### Note on the selection of candidates for a given department

In APD-S, since we focus on one academic unit (e.g, department), we would only select applicants from colleges with similar admit rate and enroll rate (this means that we can form 1 department out of those schools) for the input instance.

In [156]:
# summary of college admit rate
X_test['c_admit_rate'].describe()

count    22754.000000
mean         0.347415
std          0.238890
min          0.000000
25%          0.192000
50%          0.315000
75%          0.432000
max          1.000000
Name: c_admit_rate, dtype: float64

In [157]:
# summary of college enroll rate
X_test['c_enroll_rate'].describe()

count    22754.000000
mean         0.356386
std          0.237919
min          0.000000
25%          0.180000
50%          0.285000
75%          0.433000
max          1.000000
Name: c_enroll_rate, dtype: float64

Based on the summary of the statistics of colleges' admit and enroll rate, we would select applicants from college in the range of 25th percentile and 75th percentile.

In [158]:
X_test = X_test[(X_test['c_admit_rate'] >= np.percentile(X_test['c_admit_rate'], 25)) & \
                (X_test['c_admit_rate'] <= np.percentile(X_test['c_admit_rate'], 75)) & \
                (X_test['c_enroll_rate'] >= np.percentile(X_test['c_enroll_rate'], 25)) & \
                (X_test['c_enroll_rate'] <= np.percentile(X_test['c_enroll_rate'], 75))]

# Generate input instance for APD-S

### Specification
An input instance of APD can be characterized as $I = ([n], \{p_i, q_i |i \in [n]\}, \{B_g, g|g \in G_I\}, \{b_g, g|g \in G_E\}, \{w_{ig}, \tau_g|g \in G_P, i \in g\})$.

**Note:** 
- In this dataset, we assume all candidates are qualified for an interview and they will be automatically offered once passing it. Hence, it can be explained why we use the result of *admit* as *passing the interview* and *enroll* as *accepting the offer*.

**Input details:**
- [n] = {500, 600, 700, 800, 900, 1000}
- $p_i, q_i$ for each i $\in$ [n]
- $G_I$:
    - In APD-S, we only have single interview-related constraint that can be captured as {g, B} with g = [n] being the only group in $G_I$. So $G_I$ contains only 1 group with n candidates.
    - In reality, the dataset should miss a lot of candidates that fail to get an interview since it only includes those who qualify for an interview. In this case, we can simulate how colleges with similar acceptance rate (~20-30%) as schools in this data perform. We found [Admissions report of Oxford Law](https://www.law.ox.ac.uk/sites/files/oxlaw/ug_admissions_report_2021.pdf) and identified that we can use its application-to-interview success rate to set the cap on interview-related group g $B_g$ accordingly.
- $G_E$:
    - We have enrollment-related budget constraints {g, $b_g|g \in G_E$}. In this dataset, there are two groups inside $G_E$ which indicate candidates who are in-state and those who are out-of-state.
    - Since the data should not miss any candidate who successfully enrolls, we can set the cap on enrollment-related group g $b_g$ as the actual statistics of the dataset (of those who enroll, who are in-state applicants, who are out-of-state applicants?)
- $G_P$:
    - We capture the collection of protected groups of interest $G_P$ as the combination of the race and gender of the candidates. That means $G_P$ will have 8 groups by combining Race: {Black, Hispanic, Asian, White} and Gender: {Male, Female}.
    - We identify the target quota $\tau_g$ for protected group g using the admission statistics of universities that are known for applying Affirmative Action in their admission process. 
        + Specifically, we would use the statistics of Harvard Law School, known for its [yearly commitment to Affirmative Action in the admission/employment process](https://hr.harvard.edu/files/humanresources/files/reaffirmation_statement.pdf)
        + Based on the [demographics of Hardvard Fall 2020 applications](https://www.ilrg.com/rankings/law/view/49), we calculate the percentage of each protected group in the enrollment number and set $\tau_g$ accordingly. 
        + 0.0345, 0.0415, 0.0535, 0.252, 0.1185
    - For each candidate i $\in g$ of $G_P, w_{ig}$ (the degree of relevance of i to g) is calculated by the economic status of the candidate. We use the school's location to reflect the poverty rate - an important indicator for economic status.

### Function to generate input instance

In [159]:
def generate_input(data):
    # collection of interview-related groups
    G_I = data
    
    # collection of enrollment-related groups
    in_state = (data['resident'] == 1)
    out_of_state = (data['resident'] == 0)
    G_E = [data[in_state], data[out_of_state]]
    
    # collection of protected groups
    female_black = data[(data['gender'] == 0) & (data['black']==1)]
    female_hispanic = data[(data['gender'] == 0) & (data['hispanic'] == 1)]
    female_asian = data[(data['gender'] == 0) & (data['asian'] == 1)]
    female_white = data[(data['gender'] == 0) & (data['white'] == 1)]
    female_other = data[(data['gender'] == 0) & (data['other_race'] == 1)]
    
    male_black = data[(data['gender'] == 1) & (data['black'] == 1)]
    male_hispanic = data[(data['gender'] == 1) & (data['hispanic'] == 1)]
    male_asian = data[(data['gender'] == 1) & (data['asian'] == 1)]
    male_white = data[(data['gender'] == 1) & (data['white'] == 1)]
    male_other = data[(data['gender'] == 1) & (data['other_race'] == 1)]

    G_P = [female_black, female_hispanic, female_asian, female_white, female_other,\
           male_black, male_hispanic, male_asian, male_white, male_other] 
    
    # cap imposed on interview-related group g
    B_g = int(len(data) * 0.3705)
    
    # cap imposed on enrollment-related group g
    b_g = [len(data[in_state & (data['enroll'] == 1)]), len(data[out_of_state & (data['enroll'] == 1)])]
    
    # target quota for protected group g to achieve
    target_quota = [0.0345, 0.0415, 0.0535, 0.252, 0.1185] * 2
    tau_g = np.ceil([len(data[data['enroll'] == 1]) * quota for quota in target_quota])
    
    # if there is a difference in the sum of G_P and G_E due to rounding, randomly increase one group
    # in either G_P or G_E to balance the difference (as both cap on final enrollment)
    if sum(tau_g) > sum(b_g):
        index = np.random.randint(0, len(b_g)) 
        b_g[index] += sum(tau_g) - sum(b_g)
    elif sum(tau_g) < sum(b_g):
            index = np.random.randint(0, len(tau_g))
            tau_g[index] += sum(b_g) - sum(tau_g)
    
    # relevance of i to protected group g
    w_ig = []
    for index_g in range(len(G_P)):
        g = G_P[index_g]
        arr = []
        for index_i in range(len(g)):
            arr.append(g.iloc[index_i]['es'])
        w_ig.append(arr)
    w_ig = np.asarray(w_ig, dtype = object)
    
    return G_I, G_E, G_P, B_g, b_g, tau_g, w_ig

# Solve Linear Programming

### Justification for solving the maximin LP

The objective model is $ \max\min\limits_{g \in G_P} (\sum\limits_{i \in g} w_{ig} y_i q_i / \tau_g)$.

We can rewrite it as the following to solve:

max z

s.t $ \space \space$  z $\le \sum\limits_{i \in g} w_{ig} y_i q_i / \tau_g \space \space \space \space \space \space$ for $g \in G_P$

Other constraints will be kept as original.

### Function to solve LP

In [160]:
def solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig):
    # create model
    model = LpProblem(name='APD-S', sense = LpMaximize)

    # define decision variables
    x = [LpVariable('x' + str(i), lowBound = 0, upBound = 1) for i in range(n)]
    y = [LpVariable('y' + str(i), lowBound = 0, upBound = 1) for i in range(n)]
    z = LpVariable(name='z')

    # add objective function to the model
    model += z

    # constraints for z
    for index_g in range(len(G_P)):
        constraint = []
        g = G_P[index_g]
        for index_i in range(len(g)):
            constraint.append(w_ig[index_g][index_i]*y[g.index[index_i]]*g.iloc[index_i]['q_i']/tau_g[index_g])
        model += z <= lpSum(constraint)
    
    # constraint for (2)
    constraint = [x_i for x_i in x]
    model += lpSum(constraint) <= B_g
    
    # constraints for (3)
    for i in range(n):
        model += y[i] <= x[i]*G_I.iloc[i]['p_i']
        
    # constraints for (4) in LP
    for index_g in range(len(G_E)):
        g = G_E[index_g]
        constraint = []
        for index_i in range(len(g)):
            constraint.append(y[g.index[index_i]]*g.iloc[index_i]['q_i'])
        model += lpSum(constraint) <= b_g[index_g]
    
    # solve the model 
    model.solve(COIN_CMD(msg=0))
    
    x_opt = np.zeros(n)
    y_opt = np.zeros(n)
    
    for var in model.variables()[:n]:
        x_opt[int(str(var.name)[1:])] = round(var.varValue, 3)
    
    for var in model.variables()[n:-1]:
        y_opt[int(str(var.name)[1:])] = round(var.varValue, 3)
    
    LP_opt = model.objective.value()
    
    # return x*_i, y*_i for each candidate i
    return x_opt, y_opt, LP_opt

### Function to verify the constraints of LP

In [161]:
# verify if x* and y* of each input instance satisfy the constraints 
def verify_LP(data, n, G_E, B_g, b_g, x_opt, y_opt):
    print('Verify input instance with n =', n)

    # constraint 2
    if sum(x_opt) > B_g:
        print('Not satisfied!')
        return False

    # constraint 3
    for i in range(n):
        if y_opt[i] > x_opt[i]*data.iloc[i]['p_i']:
            print('Not satisfied!')
            return False

    # constraint 4
    for index_g in range(len(G_E)):
        g = G_E[index_g]
        for index_i in range(len(g)):
            if y_opt[g.index[index_i]]*g.iloc[index_i]['q_i'] > b_g[index_g]:
                print('Not satisfied!')
                return False
    
    # value of x*_i
    count = sum(1 for x in x_opt if x != (0))
    if count == 0:
        print('-> Value of x optimal not satisfied! \n')
            
    # if no constraint is violated 
    print('-> Every constraints has been satisfied! \n')

### Define n candidates and randomly sample data of size n from the test data

In [162]:
sizes = [500, 600, 700, 800]
input_instances = [X_test.sample(n = n).reset_index(drop = True) for n in sizes]

### Verify LP with all input instances

In [164]:
for i in range(len(input_instances)):
    # generate input instance
    data = input_instances[i]
    n = sizes[i]
    G_I, G_E, G_P, B_g, b_g, tau_g, w_ig = generate_input1(data)
              
    # output x*_i, y*_i for each i in [n]
    x_opt, y_opt, LP_opt = solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig)
    verify_LP(data, n, G_E, B_g, b_g, x_opt, y_opt)

Verify input instance with n = 500
-> Every constraints has been satisfied! 

Verify input instance with n = 600
-> Every constraints has been satisfied! 

Verify input instance with n = 700
-> Every constraints has been satisfied! 

Verify input instance with n = 800
-> Every constraints has been satisfied! 



# Implementation of algorithm 1 for the special case of APD-S

In [166]:
def algorithm_1(data, n, G_E, B_g, b_g, x_opt, y_opt):
    # remarks on RP for APD-S
    L_i = [pd.Interval(left = sum(x_opt[:i]), right = sum(x_opt[:(i+1)]), closed = 'both')\
           for i in range(n)]
    T_l = []
    T_l_prime = []
    for l in range(n):
        T_l.append(np.random.uniform(0, np.nextafter(1, np.inf), B_g))
        T_l_prime.append([l - 1 + T for T in T_l[l]])

    # set I_i = 1 if there exists at least one l in n with T'_l in L_i
    I_i = [0]*n
    for i in range(n):
        for l in range(n):
            if all(num in L_i[i] for num in T_l_prime[l]):
                I_i[i] = 1
                break
   
    # step (5) - (11):
    sigma_i = np.random.uniform(0, np.nextafter(1, np.inf), n)
    d = {i : sigma_i[i] for i in range(n)}    
    pi = list(dict(sorted(d.items(), key=lambda x: x[1], reverse=True)).keys())
    Z_i = np.zeros(n)
    sum_Zi = [0, 0]
    for l in range(n):
        i = pi[l]
        if I_i[i] == 1: # give interview to i
            p_i = data.iloc[i]['p_i']
            pass_interview = np.random.choice([True, False], size = 1, p = [p_i, 1 - p_i])
            if pass_interview:
                O_i =  np.random.binomial(1, y_opt[i]/(x_opt[i] * p_i))
                if O_i == 1: 
                    for index_g in range(len(G_E)):
                        # locate i in which group g of G_E
                        if i in G_E[index_g].index: 
                            if sum_Zi[index_g] <= b_g[index_g]:
                                sum_Zi[index_g] += 1
                                Z_i[i] = 1         
                                break
    
    return Z_i

# Verify Lemma 2: $E[Z_i] \ge (y_i^{*}.q_i)/2$ for every $i \in n$

### Function to find expected Z or E[$Z_i$]

In [167]:
def calc_E_Z(data, n, G_E, B_g, b_g, x_opt, y_opt):
    print('Verify Lemma 2 with input instance that has n =', n)
    E_Z = np.zeros(n)
    for i in range(1000):
        Z_i = algorithm_1(data, n, G_E, B_g, b_g, x_opt, y_opt)
        E_Z = np.add(E_Z, Z_i)
    E_Z = E_Z/1000
    return E_Z

### Calculate approximation ratio

In [168]:
E_ALG = []
OPT = []
for i in range(len(input_instances)):
    # generate input for RP
    data = input_instances[i]
    n = sizes[i]
    G_I, G_E, G_P, B_g, b_g, tau_g, w_ig = generate_input(data)
    x_opt, y_opt, LP_opt = solveLP(data, n, G_I, G_E, G_P, B_g, b_g, tau_g, w_ig)      
    
    # run RP 1000 times for each input instance
    E_Z = calc_E_Z(data, n, G_E, B_g, b_g, x_opt, y_opt)
    rp_val = [0] * len(G_P)
    for index_g in range(len(G_P)):
        g = G_P[index_g]
        for index_i in range(len(g)):
            rp_val[index_g] += w_ig[index_g][index_i]*E_Z[g.index[index_i]]/tau_g[index_g]
    
    E_ALG.append(min(rp_val))
    OPT.append(LP_opt)

Verify Lemma 2 with input instance that has n = 500
Case 0
Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7
Case 8
Case 9
Case 10
Case 11
Case 12
Case 13
Case 14
Case 15
Case 16
Case 17
Case 18
Case 19
Case 20
Case 21
Case 22
Case 23
Case 24
Case 25
Case 26
Case 27
Case 28
Case 29
Case 30
Case 31
Case 32
Case 33
Case 34
Case 35
Case 36
Case 37
Case 38
Case 39
Case 40
Case 41
Case 42
Case 43
Case 44
Case 45
Case 46
Case 47
Case 48
Case 49
Case 50
Case 51
Case 52
Case 53
Case 54
Case 55
Case 56
Case 57
Case 58
Case 59
Case 60
Case 61
Case 62
Case 63
Case 64
Case 65
Case 66
Case 67
Case 68
Case 69
Case 70
Case 71
Case 72
Case 73
Case 74
Case 75
Case 76
Case 77
Case 78
Case 79
Case 80
Case 81
Case 82
Case 83
Case 84
Case 85
Case 86
Case 87
Case 88
Case 89
Case 90
Case 91
Case 92
Case 93
Case 94
Case 95
Case 96
Case 97
Case 98
Case 99
Case 100
Case 101
Case 102
Case 103
Case 104
Case 105
Case 106
Case 107
Case 108
Case 109
Case 110
Case 111
Case 112
Case 113
Case 114
Case 115
Case 116
Case 

Case 917
Case 918
Case 919
Case 920
Case 921
Case 922
Case 923
Case 924
Case 925
Case 926
Case 927
Case 928
Case 929
Case 930
Case 931
Case 932
Case 933
Case 934
Case 935
Case 936
Case 937
Case 938
Case 939
Case 940
Case 941
Case 942
Case 943
Case 944
Case 945
Case 946
Case 947
Case 948
Case 949
Case 950
Case 951
Case 952
Case 953
Case 954
Case 955
Case 956
Case 957
Case 958
Case 959
Case 960
Case 961
Case 962
Case 963
Case 964
Case 965
Case 966
Case 967
Case 968
Case 969
Case 970
Case 971
Case 972
Case 973
Case 974
Case 975
Case 976
Case 977
Case 978
Case 979
Case 980
Case 981
Case 982
Case 983
Case 984
Case 985
Case 986
Case 987
Case 988
Case 989
Case 990
Case 991
Case 992
Case 993
Case 994
Case 995
Case 996
Case 997
Case 998
Case 999
Verify Lemma 2 with input instance that has n = 600
Case 0
Case 1
Case 2
Case 3
Case 4
Case 5
Case 6
Case 7
Case 8
Case 9
Case 10
Case 11
Case 12
Case 13
Case 14
Case 15
Case 16
Case 17
Case 18
Case 19
Case 20
Case 21
Case 22
Case 23
Case 24
Case 25
Cas

Case 834
Case 835
Case 836
Case 837
Case 838
Case 839
Case 840
Case 841
Case 842
Case 843
Case 844
Case 845
Case 846
Case 847
Case 848
Case 849
Case 850
Case 851
Case 852
Case 853
Case 854
Case 855
Case 856
Case 857
Case 858
Case 859
Case 860
Case 861
Case 862
Case 863
Case 864
Case 865
Case 866
Case 867
Case 868
Case 869
Case 870
Case 871
Case 872
Case 873
Case 874
Case 875
Case 876
Case 877
Case 878
Case 879
Case 880
Case 881
Case 882
Case 883
Case 884
Case 885
Case 886
Case 887
Case 888
Case 889
Case 890
Case 891
Case 892
Case 893
Case 894
Case 895
Case 896
Case 897
Case 898
Case 899
Case 900
Case 901
Case 902
Case 903
Case 904
Case 905
Case 906
Case 907
Case 908
Case 909
Case 910
Case 911
Case 912
Case 913
Case 914
Case 915
Case 916
Case 917
Case 918
Case 919
Case 920
Case 921
Case 922
Case 923
Case 924
Case 925
Case 926
Case 927
Case 928
Case 929
Case 930
Case 931
Case 932
Case 933
Case 934
Case 935
Case 936
Case 937
Case 938
Case 939
Case 940
Case 941
Case 942
Case 943
Case 944
C

Case 751
Case 752
Case 753
Case 754
Case 755
Case 756
Case 757
Case 758
Case 759
Case 760
Case 761
Case 762
Case 763
Case 764
Case 765
Case 766
Case 767
Case 768
Case 769
Case 770
Case 771
Case 772
Case 773
Case 774
Case 775
Case 776
Case 777
Case 778
Case 779
Case 780
Case 781
Case 782
Case 783
Case 784
Case 785
Case 786
Case 787
Case 788
Case 789
Case 790
Case 791
Case 792
Case 793
Case 794
Case 795
Case 796
Case 797
Case 798
Case 799
Case 800
Case 801
Case 802
Case 803
Case 804
Case 805
Case 806
Case 807
Case 808
Case 809
Case 810
Case 811
Case 812
Case 813
Case 814
Case 815
Case 816
Case 817
Case 818
Case 819
Case 820
Case 821
Case 822
Case 823
Case 824
Case 825
Case 826
Case 827
Case 828
Case 829
Case 830
Case 831
Case 832
Case 833
Case 834
Case 835
Case 836
Case 837
Case 838
Case 839
Case 840
Case 841
Case 842
Case 843
Case 844
Case 845
Case 846
Case 847
Case 848
Case 849
Case 850
Case 851
Case 852
Case 853
Case 854
Case 855
Case 856
Case 857
Case 858
Case 859
Case 860
Case 861
C

Case 668
Case 669
Case 670
Case 671
Case 672
Case 673
Case 674
Case 675
Case 676
Case 677
Case 678
Case 679
Case 680
Case 681
Case 682
Case 683
Case 684
Case 685
Case 686
Case 687
Case 688
Case 689
Case 690
Case 691
Case 692
Case 693
Case 694
Case 695
Case 696
Case 697
Case 698
Case 699
Case 700
Case 701
Case 702
Case 703
Case 704
Case 705
Case 706
Case 707
Case 708
Case 709
Case 710
Case 711
Case 712
Case 713
Case 714
Case 715
Case 716
Case 717
Case 718
Case 719
Case 720
Case 721
Case 722
Case 723
Case 724
Case 725
Case 726
Case 727
Case 728
Case 729
Case 730
Case 731
Case 732
Case 733
Case 734
Case 735
Case 736
Case 737
Case 738
Case 739
Case 740
Case 741
Case 742
Case 743
Case 744
Case 745
Case 746
Case 747
Case 748
Case 749
Case 750
Case 751
Case 752
Case 753
Case 754
Case 755
Case 756
Case 757
Case 758
Case 759
Case 760
Case 761
Case 762
Case 763
Case 764
Case 765
Case 766
Case 767
Case 768
Case 769
Case 770
Case 771
Case 772
Case 773
Case 774
Case 775
Case 776
Case 777
Case 778
C

$\min\limits_{g \in G_P} (\sum\limits_{i \in G} E[Z_i].w_{ig} / \tau_g)$

In [169]:
E_ALG

[0.056417, 0.022841999999999998, 0.0111595, 0.021995999999999998]

$\min\limits_{g \in G_P} (\sum\limits_{i \in G} y^{*}_i q_i w_{ig} / \tau_g)$

In [170]:
OPT

[0.0032904, 0.0001944, 0.0007884, 0.0015876]

# Greedy algorithm

# Uniform algorithm

# Visualize the confidence interval of the Approximation Ratio for each algorithm